# Import Necessary Libraries

In [1]:
# Import standard libraries
import sys
import os
import pandas as pd
from sqlalchemy import event
from sqlalchemy.engine import Engine
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, os.pardir))

if project_root not in sys.path:
    sys.path.append(project_root)

from app import create_app, db
from app.utils import create_returns_df
from app.models import SeriesGroup, TimeSeriesType, TimeSeries, Keyword, DataPoint
from app.series import SeriesSearcher

DATABASE_OPTS = ['development', 'testing', 'production']

app = create_app(DATABASE_OPTS[0])
app_context = app.app_context()
app_context.push()

## Data from Any Source

In [2]:
returns = create_returns_df()
display(returns.head())

,GTO,KHU,GZS,WKK,XUD
1940-09-30,-0.125609,-0.079461,0.184972,-0.191892,-0.019459
1940-10-31,0.108064,0.023717,0.057545,-0.108832,0.004749
1940-11-30,0.054237,0.202111,-0.114830,0.054499,0.016764
1940-12-31,-0.054445,0.211579,-0.001270,-0.043751,0.075742
1941-01-31,-0.017678,-0.000138,-0.033495,0.013269,0.025190


## Transform DataFrame into TimeSeries
- By default, the of the timeseries is equal to the column name
- The code of the timeseries must be passed
- You can pass the frequency of the timeseries (M for monthly, D for daily, etc...)
- You can pass the default delta type (pct of abs difference)
- You can pass description of the timeseries
- You can pass keywords for each of the timeseries

For instance:

In [3]:
mult_time_series = TimeSeries.from_dataframe(returns, code=list(returns.columns))
mult_time_series

[TimeSeries(name=GTO, code=GTO, len=1000, freq=None, delta=pct),
 TimeSeries(name=KHU, code=KHU, len=1000, freq=None, delta=pct),
 TimeSeries(name=GZS, code=GZS, len=1000, freq=None, delta=pct),
 TimeSeries(name=WKK, code=WKK, len=1000, freq=None, delta=pct),
 TimeSeries(name=XUD, code=XUD, len=1000, freq=None, delta=pct)]

In [4]:
adj_ret_description = lambda x: f"Returns adjusted by dividends of stock {x}"

mult_time_series = TimeSeries.from_dataframe(
    returns, 
    name=[c + " Adj Returns" for c in list(returns.columns)],
    code=[c + "AR" for c in list(returns.columns)],
    description=[adj_ret_description(c) for c in list(returns.columns)],
    time_frequency='M'
)
mult_time_series

[TimeSeries(name=GTO Adj Returns, code=GTOAR, len=1000, freq=M, delta=pct),
 TimeSeries(name=KHU Adj Returns, code=KHUAR, len=1000, freq=M, delta=pct),
 TimeSeries(name=GZS Adj Returns, code=GZSAR, len=1000, freq=M, delta=pct),
 TimeSeries(name=WKK Adj Returns, code=WKKAR, len=1000, freq=M, delta=pct),
 TimeSeries(name=XUD Adj Returns, code=XUDAR, len=1000, freq=M, delta=pct)]

## Transform TimeSeries into DataFrame

Transform a single one:

In [5]:
first_time_series = mult_time_series[0]
first_time_series.to_dataframe()

,GTO Adj Returns
date,
1940-09-30,-0.125609
1940-10-31,0.108064
1940-11-30,0.054237
1940-12-31,-0.054445
1941-01-31,-0.017678
...,...
2023-08-31,-0.028455
2023-09-30,-0.050625
2023-10-31,-0.079678


Transform multiple together:

In [6]:
first_time_series = mult_time_series[0]
second_time_series = mult_time_series[1]

display(TimeSeries.join_timeseries_to_dataframe([first_time_series, second_time_series], how='outer'))

display(first_time_series.join_with_other_timeseries_to_dataframe(second_time_series, how='outer'))

,GTO Adj Returns,KHU Adj Returns
date,,
1940-09-30,-0.125609,-0.079461
1940-10-31,0.108064,0.023717
1940-11-30,0.054237,0.202111
1940-12-31,-0.054445,0.211579
1941-01-31,-0.017678,-0.000138
...,...,...
2023-08-31,-0.028455,0.048011
2023-09-30,-0.050625,-0.026853
2023-10-31,-0.079678,-0.115953


,GTO Adj Returns,KHU Adj Returns
date,,
1940-09-30,-0.125609,-0.079461
1940-10-31,0.108064,0.023717
1940-11-30,0.054237,0.202111
1940-12-31,-0.054445,0.211579
1941-01-31,-0.017678,-0.000138
...,...,...
2023-08-31,-0.028455,0.048011
2023-09-30,-0.050625,-0.026853
2023-10-31,-0.079678,-0.115953


## How to save in the database?

In [7]:
first_time_series = mult_time_series[0]
display(first_time_series)
display(first_time_series.description)

TimeSeries(name=GTO Adj Returns, code=GTOAR, len=1000, freq=M, delta=pct)

'Returns adjusted by dividends of stock GTO'

In [8]:
first_time_series.save()

## How to save multiple TimeSeries objects at once?

In [9]:
TimeSeries.save_all(mult_time_series)

## How to add keywords to TimeSeries?

In [10]:
returns_one_asset = create_returns_df(n_assets=1, seed=1)
display(returns_one_asset)

,FLM
1940-09-30,0.022150
1940-10-31,0.003604
1940-11-30,0.159087
1940-12-31,-0.169918
1941-01-31,0.226510
...,...
2023-08-31,0.171804
2023-09-30,0.129719
2023-10-31,0.188198
2023-11-30,-0.175475


In [11]:
ts_returns_one_asset = TimeSeries.from_dataframe(
    returns_one_asset,
    name="FLM Not Adjusted Returns",
    code="FLMNAR",
    description="Not adjusted returns for FLM stock",
    time_frequency='M',
)
ts_returns_one_asset.add_keyword(['FLM', 'Returns', 'Industry', 'Stock', 'Consultant', 'Mexico'])
ts_returns_one_asset.save()

/Users/fernandorochacorreaurbano/Desktop/investment-portfolio/app/models.py:159: SAWarning: Object of type <TimeSeries> not in session, add operation along 'Keyword.series' won't proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  keyword = session.query(Keyword).filter_by(word=keyword_word).first()
/Users/fernandorochacorreaurbano/Desktop/investment-portfolio/app/models.py:305: SAWarning: Object of type <TimeSeries> not in session, add operation along 'Keyword.series' won't proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation.)
  time_series_with_same_name = session.query(TimeSeries).filter_by(name=self.name).first()


## How to sarch for TimeSeries already saved?

In [12]:
SeriesSearcher.search('Industry')

,type,id,name,code,description,keywords
0,TimeSeries,11,FLM Not Adjusted Returns,FLMNAR,Not adjusted returns for FLM stock,"FLM, Returns, Industry, Stock, Consultant, Mexico"


In [13]:
SeriesSearcher.search('Returns')

,type,id,name,code,description,keywords
0,TimeSeries,6,GTO Adj Returns,GTOAR,Returns adjusted by dividends of stock GTO,
1,TimeSeries,7,KHU Adj Returns,KHUAR,Returns adjusted by dividends of stock KHU,
2,TimeSeries,8,GZS Adj Returns,GZSAR,Returns adjusted by dividends of stock GZS,
3,TimeSeries,9,WKK Adj Returns,WKKAR,Returns adjusted by dividends of stock WKK,
4,TimeSeries,10,XUD Adj Returns,XUDAR,Returns adjusted by dividends of stock XUD,
5,TimeSeries,11,FLM Not Adjusted Returns,FLMNAR,Not adjusted returns for FLM stock,"FLM, Returns, Industry, Stock, Consultant, Mexico"


In [14]:
SeriesSearcher.search('Returns Industry')

,type,id,name,code,description,keywords
0,TimeSeries,6,GTO Adj Returns,GTOAR,Returns adjusted by dividends of stock GTO,
1,TimeSeries,7,KHU Adj Returns,KHUAR,Returns adjusted by dividends of stock KHU,
2,TimeSeries,8,GZS Adj Returns,GZSAR,Returns adjusted by dividends of stock GZS,
3,TimeSeries,9,WKK Adj Returns,WKKAR,Returns adjusted by dividends of stock WKK,
4,TimeSeries,10,XUD Adj Returns,XUDAR,Returns adjusted by dividends of stock XUD,
5,TimeSeries,11,FLM Not Adjusted Returns,FLMNAR,Not adjusted returns for FLM stock,"FLM, Returns, Industry, Stock, Consultant, Mexico"
